In [1]:
# Importing libraries
from bs4 import BeautifulSoup
#from multiprocessing import Process, Manager
import pandas as pd
import requests
import urllib.request

#from datetime import datetime
#, sys, os, time, json, re, random, ast



In [21]:
urls = []

# User Inputs
page_number=1
topic='cyber+security' # Select a topic from the list available here: https://www.ted.com/topics
f = open('TED_Talk_'+topic+'_URLs.txt', 'w')

while True:
    
  res  =  requests.get("https://www.ted.com/talks?page="+str(page_number)+"&sort=newest&topics[]="+topic)

  soup = BeautifulSoup(res.text)

  e=soup.find('div', id='browse-results')


  string_end="Sorry. We couldn't find a talk quite like that." 

  if e is None:
    break
  elif e.find('div', class_='h3 m2') is not None:
    if e.find('div', class_='h3 m2').text==string_end:
      break
  else:
    soup = BeautifulSoup(res.text)
    e=soup.find('div', id='browse-results')

    for elem in e.find_all("h4", class_="h9 m5 f-w:700"):
      urls.append(elem.find('a')['href'])
    page_number+=1


f.write('\n'.join(urls))
f.close()
